<a href="https://colab.research.google.com/github/steinhaug/stable-diffusion/blob/main/BLIP-2/blip2_vicuna_instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup - install and prepare models

In [ ]:
#@markdown Clone LAVIS, install dependencies
%cd /content
!git clone https://github.com/salesforce/LAVIS.git
%cd LAVIS
!wget https://raw.githubusercontent.com/steinhaug/stable-diffusion/main/pyscripts/file_manipulation.py -O fmnip.py
!pip install -e .
!mkdir model

In [ ]:
#@markdown Download vicuna13b weights used in replicate app
%cd /content/LAVIS/model
!wget https://storage.googleapis.com/replicate-weights/blip2-instruct-vicuna13b/pretrained_checkpoint.pt
!wget https://storage.googleapis.com/replicate-weights/blip2-instruct-vicuna13b/vicuna-13b-16fp.tensors
%cd /content/LAVIS

In [ ]:
#@markdown patch the yaml file
from IPython.display import clear_output
from fmnip import commentOutLinesStartingWith, insertInfrontWhenMatched
filename = "/content/LAVIS/lavis/configs/models/blip2/blip2_instruct_vicuna13b.yaml"
commentOutLinesStartingWith(filename, ['pretrained:','llm_model:'], '  # ')
insertInfrontWhenMatched(filename, [('#   pretrained','  pretrained: "./model/pretrained_checkpoint.pt"' + "\n",-1), ('#   llm_model','  llm_model: "./model/vicuna-13b-16fp.tensors"' + "\n",-1)] )
clear_output(); print('Patched!')

### Inference - Load model

This is not working, anyone have any idea how to make this work ?

In [ ]:
import torch
from PIL import Image
# setup device to use
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
# load sample image
raw_image = Image.open("./docs/_static/Confusing-Pictures.jpg").convert("RGB")
display(raw_image.resize((596, 437)))

In [ ]:
from lavis.models import load_model_and_preprocess
# loads InstructBLIP model
model, vis_processors, _ = load_model_and_preprocess(name="blip2_vicuna_instruct", model_type="vicuna13b", is_eval=True, device=device)
# prepare the image
image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)

In [ ]:
model.generate({"image": image, "prompt": "What is unusual about this image?"})

In [ ]:
model.generate({"image": image, "prompt": "Write a short description for the image."})

In [ ]:
model.generate({"image": image, "prompt": "Write a detailed description."})

In [ ]:
model.generate({"image": image, "prompt":"Describe the image in details."}, use_nucleus_sampling=True, top_p=0.9, temperature=1)